# The HouseBlend example

In [10]:
import os
import sys
import time
import matplotlib.pyplot as plt
import numpy as np

module_path = os.path.abspath(os.path.join('../HouseBlend'))
if module_path not in sys.path:
    sys.path.append(module_path)
import HouseBlend as hb

## Setup an example test case
If using for real, first create the contacts.xlsx workbook following the template and leave the test parameter as False (default value).

### User inputs
**n_periods** <br>
The total number of periods you wish to schedule including historic and future. 
Set this to None to have the minimum number of periods required for everyone to meet everyone else

**current_period** <br>
The period number of the first future period to schedule. 
For first ever usage, this should be set to 1 (i.e. you need to schedule from period 1 forwards).
Values > 1 will import previous periods from schedule.xlsx and treat these as having happened and therefore fixed.

**parent_dir** <br>
The folder path containing the input / output files. If it does not exist, it will be created.

In [11]:
n_periods = None  # total number of periods to schedule. Set to None for minimum number to satisfy all possible meetings.

current_period = 1  # the period from which to schedule. Set to 1 for first usage.

parent_fullpath = os.path.join('../', 'example')  # the folder containing the input files (contacts.xlsx). Outputs will be saved here.

## Import input files or create test participants
Use import_contacts_df with test set to an integer to create a new test group of n participants. For real usage, remove test parameter (default is false) to use on a defined contacts.xlsx input file.

Import historic schedule if it exists - required for historic inclusion e.g. if current_period>1, setting test=True deletes any previous schedule!

In [12]:
contacts, dates, availability = hb.import_contacts_df(folderpath=parent_fullpath, test=6)

schedule, bool_schedule = hb.import_schedules(folderpath=parent_fullpath, test=True)

contacts.head()

Deleting previous contacts file
Creating new contact directory of 6 length for testing
Deleting previous schedule files
No existing schedule found. Ensure current_period is set to 1.


,email,Assistant,Assistant email
Person,,,
Christine Delgado,ChristineDelgado@email.com,Christine Delgado,ChristineDelgado@email.com
Melissa Lewis,MelissaLewis@email.com,Melissa Lewis,MelissaLewis@email.com
Sarah Cole,SarahCole@email.com,Sarah Cole,SarahCole@email.com
Sean Miller,SeanMiller@email.com,Sean Miller,SeanMiller@email.com
Alexander Yoder,AlexanderYoder@email.com,Alexander Yoder,AlexanderYoder@email.com


In [13]:
dates

,Start Date,End Date
Period,,
1,2025-05-18,2025-06-01
2,2025-06-01,2025-06-15
3,2025-06-15,2025-06-29
4,2025-06-29,2025-07-13
5,2025-07-13,2025-07-27


## Run schedule optimisation

In [14]:
bool_schedule = hb.run_schedule_optimisation(contacts, bool_schedule, n_periods, availability)

## Create readable meeting schedule


In [18]:
meeting_schedule = hb.generate_meeting_schedule(contacts, bool_schedule, dates, folderpath=parent_fullpath, save=False)
meeting_schedule

,Period 1,Period 2,Period 3,Period 4,Period 5
Person,,,,,
Christine Delgado,Alexander Yoder,Sean Miller,Sarah Cole,Melissa Lewis,Abigail Hurst
Melissa Lewis,Sarah Cole,Alexander Yoder,Abigail Hurst,Christine Delgado,Sean Miller
Sarah Cole,Melissa Lewis,Abigail Hurst,Christine Delgado,Sean Miller,Alexander Yoder
Sean Miller,Abigail Hurst,Christine Delgado,Alexander Yoder,Sarah Cole,Melissa Lewis
Alexander Yoder,Christine Delgado,Melissa Lewis,Sean Miller,Abigail Hurst,Sarah Cole
Abigail Hurst,Sean Miller,Sarah Cole,Melissa Lewis,Alexander Yoder,Christine Delgado


## Create a single period's list of meetings - ready for mail merge export

In [20]:
all_meetings_period = hb.period_meeting_list(contacts, bool_schedule, 1, save=False, folderpath=parent_fullpath)
all_meetings_period

,Person 1,Person 2
0,Sarah Cole,Melissa Lewis
1,Alexander Yoder,Christine Delgado
2,Abigail Hurst,Sean Miller


## Export for mailmerge

In [23]:
hb.export_for_mailmerge(
    contacts,
    bool_schedule,
    1,
    dates,
    save=True,
    folderpath=parent_fullpath,
    save_prefix='HouseBlend',
    subject=None)

,Person 1,Person 2,Person 1 first name,Person 2 first name,Person 1 assistant,Person 2 assistant,Person 1 assistant first name,Person 2 assistant first name,Start Date,End Date,Start Date Long,End Date Long,All emails,Mailto Link,Email Type,mailto
0,Sarah Cole,Melissa Lewis,Sarah,Melissa,Sarah Cole,Melissa Lewis,Sarah,Melissa,18/05/2025,01/06/2025,Sunday 18th May 2025,Sunday 1st June 2025,<SarahCole@email.com>; <MelissaLewis@email.com...,mailto:SarahCole@email.com;MelissaLewis@email....,Person 1 email,SarahCole@email.com
1,Alexander Yoder,Christine Delgado,Alexander,Christine,Alexander Yoder,Christine Delgado,Alexander,Christine,18/05/2025,01/06/2025,Sunday 18th May 2025,Sunday 1st June 2025,<AlexanderYoder@email.com>; <ChristineDelgado@...,mailto:AlexanderYoder@email.com;ChristineDelga...,Person 1 email,AlexanderYoder@email.com
2,Abigail Hurst,Sean Miller,Abigail,Sean,Abigail Hurst,Sean Miller,Abigail,Sean,18/05/2025,01/06/2025,Sunday 18th May 2025,Sunday 1st June 2025,<AbigailHurst@email.com>; <SeanMiller@email.co...,mailto:AbigailHurst@email.com;SeanMiller@email...,Person 1 email,AbigailHurst@email.com
3,Sarah Cole,Melissa Lewis,Sarah,Melissa,Sarah Cole,Melissa Lewis,Sarah,Melissa,18/05/2025,01/06/2025,Sunday 18th May 2025,Sunday 1st June 2025,<SarahCole@email.com>; <MelissaLewis@email.com...,mailto:SarahCole@email.com;MelissaLewis@email....,Person 2 email,MelissaLewis@email.com
4,Alexander Yoder,Christine Delgado,Alexander,Christine,Alexander Yoder,Christine Delgado,Alexander,Christine,18/05/2025,01/06/2025,Sunday 18th May 2025,Sunday 1st June 2025,<AlexanderYoder@email.com>; <ChristineDelgado@...,mailto:AlexanderYoder@email.com;ChristineDelga...,Person 2 email,ChristineDelgado@email.com
5,Abigail Hurst,Sean Miller,Abigail,Sean,Abigail Hurst,Sean Miller,Abigail,Sean,18/05/2025,01/06/2025,Sunday 18th May 2025,Sunday 1st June 2025,<AbigailHurst@email.com>; <SeanMiller@email.co...,mailto:AbigailHurst@email.com;SeanMiller@email...,Person 2 email,SeanMiller@email.com
6,Sarah Cole,Melissa Lewis,Sarah,Melissa,Sarah Cole,Melissa Lewis,Sarah,Melissa,18/05/2025,01/06/2025,Sunday 18th May 2025,Sunday 1st June 2025,<SarahCole@email.com>; <MelissaLewis@email.com...,mailto:SarahCole@email.com;MelissaLewis@email....,Person 1 assistant email,SarahCole@email.com
7,Alexander Yoder,Christine Delgado,Alexander,Christine,Alexander Yoder,Christine Delgado,Alexander,Christine,18/05/2025,01/06/2025,Sunday 18th May 2025,Sunday 1st June 2025,<AlexanderYoder@email.com>; <ChristineDelgado@...,mailto:AlexanderYoder@email.com;ChristineDelga...,Person 1 assistant email,AlexanderYoder@email.com
8,Abigail Hurst,Sean Miller,Abigail,Sean,Abigail Hurst,Sean Miller,Abigail,Sean,18/05/2025,01/06/2025,Sunday 18th May 2025,Sunday 1st June 2025,<AbigailHurst@email.com>; <SeanMiller@email.co...,mailto:AbigailHurst@email.com;SeanMiller@email...,Person 1 assistant email,AbigailHurst@email.com
9,Sarah Cole,Melissa Lewis,Sarah,Melissa,Sarah Cole,Melissa Lewis,Sarah,Melissa,18/05/2025,01/06/2025,Sunday 18th May 2025,Sunday 1st June 2025,<SarahCole@email.com>; <MelissaLewis@email.com...,mailto:SarahCole@email.com;MelissaLewis@email....,Person 2 assistant email,MelissaLewis@email.com
